## Angelo above here, Ollie below here

In [12]:
train_data_path = "./data/train.csv"

# Reading CSV file
df = pd.read_csv(train_data_path)

### KNN all variables, aside from num_supermarkets and binaries

In [13]:
# WHAT TO DO ABOUT FLOOR WITH KNN???
vars_to_impute = ['num_rooms', 'num_baths', 'square_meters', 'year_built', 'num_crimes', 'price']
impute_df = df[vars_to_impute]

imputer = KNNImputer(n_neighbors=7)
imputed_data = imputer.fit_transform(impute_df)
imputed_df = pd.DataFrame(imputed_data, columns = impute_df.columns)

for var in vars_to_impute:
    df[var] = imputed_df[var]

#### Imputing neighborhood based on the number of crimes

In [15]:
crimes_by_neighborhood = df.groupby('neighborhood')['num_crimes'].mean().to_dict()

# Function to be used in the imputation process
def find_closest_key(target, dictionary):
    closest_key = None
    closest_difference = float('inf')

    for key, value in dictionary.items():
        difference = abs(target - value)
        if difference < closest_difference:
            closest_key = key
            closest_difference = difference

    return closest_key

# Imputing neighborhood based on number of crimes
def impute_neighborhood(row):
    crimes = row['num_crimes']
    neighborhood = row['neighborhood']
    if pd.isnull(neighborhood):
        return find_closest_key(crimes, crimes_by_neighborhood)
    else:
        return neighborhood
    
df['neighborhood'] = df.apply(impute_neighborhood, axis=1)

#### Imputing number of supermarkets with mean of the neighborhood

In [19]:
neighborhood_means = df.groupby('neighborhood')['num_supermarkets'].transform('mean')
print(neighborhood_means)

0       2.036496
1       2.051613
2       1.971429
3       2.051613
4       1.884615
          ...   
7995    2.036496
7996    2.055556
7997    2.036496
7998    1.884615
7999    2.055556
Name: num_supermarkets, Length: 8000, dtype: float64
